In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv
import os
import time
from datetime import datetime, timedelta
import pandas as pd
import keybert
from nltk.corpus import stopwords
import re 


c:\Users\Marcell Suyanto\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
load_dotenv()
user = os.getenv("UID")
password = os.getenv("PASSWORD")

#JOB TYPE: 
# Graduate 
# Internsip 
# Temporary 
# Summer
job_type = "Internship"

<h2>Logging into the Job Portal</h2>

In [3]:
driver = webdriver.Chrome()
driver.get("https://www.cedars.hku.hk/netjobs")
main_page = driver.current_window_handle

# Click on Student Login
student_login = driver.find_element(By.XPATH, "//a[text()='HKU Student']")
student_login.click()

#Input email and log in
email_input = WebDriverWait(driver, timeout=10).until(
    EC.presence_of_element_located((By.ID, "email"))
)
login_button = driver.find_element(By.ID, "login_btn")

email_input.send_keys(user)
login_button.click()

#Input password
password_input = WebDriverWait(driver, timeout=10).until(
    EC.presence_of_element_located((By.ID, "passwordInput"))
)
sign_in_button = driver.find_element(By.ID, "submitButton")
password_input.send_keys(password)
sign_in_button.click()

#Trust Page
continue_button = WebDriverWait(driver, timeout=10).until(
    EC.presence_of_element_located((By.ID, "idSIButton9"))
)
continue_button.click()
#Stay Singed in page
stay_button = WebDriverWait(driver, timeout=10).until(
    EC.visibility_of_element_located((By.XPATH, "//input[@type='submit' and @id='idSIButton9']"))
)
stay_button.click()

check_box = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, "//input[@type='checkbox']"))
)
check_box.click()
agree_btn = WebDriverWait(driver,10).until(
    EC.element_to_be_clickable((By.ID, "btn-agree"))
)
agree_btn.click()

time.sleep(1)

job_type_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, f"//a[text()='{job_type} (']"))
)
job_type_button.click()

time.sleep(1)

job_type_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, f"//a[text()='{job_type} (']"))
)
job_type_button.click()

In [4]:

yesterday = datetime.now() - timedelta(1)
yesterday = datetime.strftime(yesterday, '%Y-%m-%d')
yesterday

'2025-07-29'

In [5]:
def clean_data(data):
    # data: list of strings corresponding to job details
    for i in range(len(data)):
        data[i] = data[i].split(':\n')
    return data

def get_data(driver):
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))

    for window_handle in driver.window_handles:
        if window_handle != main_page:
            driver.switch_to.window(window_handle)
            break
    
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//div[@id='content']"))
    )
    extract_data = driver.find_elements(By.XPATH, "//div[@id='content']//div[contains(@class, 'crow')]")
    data = [i.text for i in extract_data if i.text.strip()]
    
    driver.close()
    driver.switch_to.window(main_page)
    return data

details = []
jobs = driver.find_elements(By.XPATH, "//table[@id='search_jobs']/tbody/tr")

for i, job in enumerate(jobs):
    job.click()
    data = clean_data(get_data(driver))
    details.append(data)

driver.close()

In [6]:
details[2]

[['Job ID', 'G2500126'],
 ['Posting Date', '2025-07-28'],
 ['Employment Type', 'Internship'],
 ['Employment Mode', 'Part-time'],
 ['Company Name (Eng)', 'RecruitFirst Limited'],
 ['Nature of Business',
  'Human Resources Consulting / Recruitment Agency / Training'],
 ['Origin of Company Ownership', 'Singapore'],
 ['No. of Employees', '10-50 employees'],
 ['Position Offered', 'HR Recruitment Internship'],
 ['No. of Vacancies', '3'],
 ['Job Nature', 'Human Resources / Training / Recruitment'],
 ['Job Description',
  "As a Recruitment Intern you will learn: \nInterpersonal skills and picking up on different ques as you interact with different candidates\nProblem-solving skills to address candidates' issues and conflicts\nHow to do candidate mapping and market research from our structured training sessions\nPre-interview assessment techniques with on-the-job coaching\nWinning practices with our hands-on mentorship program \nOur Industry Practices: \nBanking & Finance\nHR & Research\nFMCG\n

In [7]:
job_details = dict()
for job in details:
    for detail in job:
        if detail[0] not in job_details:
            job_details[detail[0]] = 1
        else:
            job_details[detail[0]] += 1

job_details


{'Job ID': 20,
 'Posting Date': 20,
 'Employment Type': 20,
 'Employment Mode': 20,
 'Company Name (Eng)': 20,
 'Nature of Business': 20,
 'Origin of Company Ownership': 20,
 'No. of Employees': 20,
 'Position Offered': 20,
 'No. of Vacancies': 20,
 'Job Nature': 20,
 'Job Description': 20,
 'Basic Salary': 20,
 'Work Location': 20,
 'Work Mode': 20,
 'Fields of Study Required': 20,
 'Level of Award ': 20,
 'No. of working hours per day': 20,
 'No. of working days per week': 20,
 'Closing Date for Application': 20,
 'Application should be submitted by/via:': 20,
 'Email': 13,
 'Application Documents Required': 20,
 'Company Name (Chi)': 8,
 'Online via NETjobs:': 6,
 'Application should be addressed to': 8,
 'Other Benefits': 4,
 'Online Application at company website': 5,
 'Mail': 1,
 'Phone': 1,
 'Fax': 1,
 'Others': 1}

In [8]:
jobs_data = []
for job in details:
    jobs_dict = dict()
    for detail in job:
        if len(detail) == 2: 
            jobs_dict[detail[0]] = detail[1]
        elif len(detail) > 2:  
            jobs_dict[detail[0]] = ' '.join(detail[1:])
        elif len(detail) == 1:
            if detail[0] not in jobs_dict:
                jobs_dict[detail[0]] = 1
            else:
                jobs_dict[detail[0]] += 1
    jobs_data.append(jobs_dict)
jobs_df = pd.DataFrame(jobs_data)

In [9]:
# jobs_df.to_json("data.json", orient="records", indent=4)

jobs_df.columns

Index(['Job ID', 'Posting Date', 'Employment Type', 'Employment Mode',
       'Company Name (Eng)', 'Nature of Business',
       'Origin of Company Ownership', 'No. of Employees', 'Position Offered',
       'No. of Vacancies', 'Job Nature', 'Job Description', 'Basic Salary',
       'Work Location', 'Work Mode', 'Fields of Study Required',
       'Level of Award ', 'No. of working hours per day',
       'No. of working days per week', 'Closing Date for Application',
       'Application should be submitted by/via:', 'Email',
       'Application Documents Required', 'Company Name (Chi)',
       'Online via NETjobs:', 'Application should be addressed to',
       'Other Benefits', 'Online Application at company website', 'Mail',
       'Phone', 'Fax', 'Others'],
      dtype='object')

In [10]:
print(jobs_df['Job Description'].iloc[19])

"""
About Us
Role Overview / Job Responsibilities / Job Overview / Job Duties / What we offer / Job Description / Key Responsibilities
Key Responsibilities
Requirements/Qualifications
"""

Job Description
Support the Client Services Management team in meeting clients’ requests and demands.  
Support Client Services Management Team's everyday deliverables expected from this team. Types of support will include scheduling of meeting, producing PowerPoints, organizing client help desk, track outstanding actions, participating at clients’ helpdesks and internally liaise with various teams to respond to client’s requests and enquiries.
 Understand the internal SOP and SLA to follow through outstanding actions within internal teams.
Actively work with internal teams, to ensure clients’ requests and demands are met.
Ensure quality written and verbal communication response is presented to the internal teams and external clients.
Prepare presentations and gather adhoc reporting requests on time
Investigate, respond & proactively resolve escalated queries both internal/external in a timely manner and where required inform the CSM
Proactively identify and action opportunities for pr

'\nAbout Us\nRole Overview / Job Responsibilities / Job Overview / Job Duties / What we offer / Job Description / Key Responsibilities\nKey Responsibilities\nRequirements/Qualifications\n'

In [11]:
def clean_desc(desc):
    desc = re.split(r'', desc)
    return desc

In [17]:
def get_tags(input_df:pd.DataFrame, model:keybert) -> list:
    tags = []
    job_nature = input_df['Job Nature'].replace('/', ' ') 
    #job_desc = clean_desc(input_df['Job Description'])
    job_desc = input_df['Job Description']
    pos_offered = input_df['Position Offered'] 
    nature_of_business = input_df['Nature of Business'] 

    tags.append(model.extract_keywords(job_desc, stop_words=stopwords.words('english')))
    return tags

In [13]:
model = keybert.KeyBERT()

In [18]:
input_data = jobs_df[['Job ID', 'Job Nature', 'Job Description', 'Position Offered', 'Nature of Business']]
print(input_data.iloc[1]['Job Description'])
print(get_tags(input_data.iloc[1], model))

Position: Coral Programme Specialist
Company: Meta Dot Limited
Location: San Po Kong
Job Type: Part-Time
Job Highlights Engage in a transformative Coral STEAM Programme
Lead in scientific research, tech implementation, and engineering solutions
Foster educational and conservation impact in the community
Job Description Are you passionate about coral conservation, technology, and education? Join us as a Coral Programme Specialist and make a real difference in both the environment and the community.

Meta Dot Limited is seeking a dedicated Coral Programme Specialist to lead and contribute to our innovative Coral STEAM Programme, an initiative that intertwines Science, Technology, Engineering, Art, and Mathematics (STEAM) to explore and conserve marine ecosystems, particularly focusing on corals.
Responsibilities 1. Scientific Research on Coral Conduct and lead research on coral ecology, threats, and conservation strategies.
Develop and implement educational materials for students and the

Job ID  
Company Name  
Nature of Business  
Position Offered  
Job Nature
Job Description

Get hastags and concatenate the vector embeddings  
Store each embedding to its respective job ID

Data types: NULL INTEGER REAL TEXT BLOB

In [15]:
#Set up a connection
# conn = sqlite3.connect('jobs.db')
# cursor = conn.cursor()
# create_table_query = """
# CREATE TABLE IF NOT EXISTS jobs (
#     JobID TEXT PRIMARY KEY,
#     PostingDate TEXT,
#     EmploymentType TEXT,
#     EmplymentMode TEXT,
#     CompanyNameEng TEXT,
#     CompanyNameChi TEXT, 
#     NatureOfBusiness TEXT,
#     OriginOfCompanyOwnership TEXT,
#     NoOfEmployees TEXT,
#     PositionOffered TEXT,
#     NoOfVacancies TEXT,
#     JobNature TEXT,
#     JobDescription TEXT,
#     BasicSalary TEXT,
#     WorkLocatin TEXT, 
#     WorkMode TEXT,
#     FieldsOfStudyRequired TEXT,
#     LevelOfAward TEXT,
#     NoOfWorkingHoursPerDay INTEGER,
#     NoOfWorkingHoursPerWeek INTEGER,
#     ClosingDataForApplication TEXT,
#     ApplicationShouldBeSubmittedVia TEXT,
#     ApplicationDocumentsRequired TEXT
# );
# """

# cursor.execute(create_table_query)

In [16]:


import nbformat

# Load the notebook
with open('get_jobs.ipynb') as f:
    notebook = nbformat.read(f, as_version=4)

# Extract code cells
code_cells = [cell.source for cell in notebook.cells if cell.cell_type == 'code']

# Write to a .py file
with open('output_script.py', 'w') as f:
    for cell in code_cells:
        f.write(cell + '\n\n')
